#### Version 2 of Sparisity Comparison
This program copies the RadiXNet implementation from RadixNet.ipynb, but instead uses it to create multiple networks with differing sparsities to compare the difference in accuracies between sparsities
Is much neater and more specified to investigating the models with sparsity >0.9 that have irregular training

## Imports

In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pickle
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import tensorflow_model_optimization as tfmot
import RadixNetFuncs
import matlab.engine
import math
import networkx as nx
from ast import literal_eval

2023-08-11 13:40:48.933065: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-11 13:40:49.739740: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-11 13:40:49.889855: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-11 13:40:49.889873: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

### Read data

In [2]:
data = pd.read_csv("MNIST/mnist_train.csv") #mnist train
test = pd.read_csv("MNIST/mnist_test.csv") #mnist test
data = np.array(data)
np.random.seed(100)
np.random.shuffle(data) 

data_train = data.T
Y_train = np.array(data_train[0])
X_train = np.array(data_train[1:]).T
X_train = X_train / 255

test = np.array(test)
data_test = test.T
Y_test = np.array(data_test[0])
X_test = np.array(data_test[1:]).T
X_test = X_test / 255

# Create different models based on sparsity

In [3]:
# desired_network_structure = [784, 300, 100, 10]
# factors = list(range(1,11))
# permutations = RadixNetFuncs.generate_permutations(desired_network_structure,factors)
# print(len(permutations))

3711


In [ ]:
# topologies = RadixNetFuncs.build_radix_nets(permutations,desired_network_structure)

In [4]:
desired = [784,300,100,10]
sparsemodels, greater90 = RadixNetFuncs.findSimpleModels(desired)
print(len(greater90))

426


In [11]:
myKeys = list(greater90.keys())
myKeys.sort()
#print(myKeys)
models = []
sampleSize = 20
sparsities = []
rstructures = []
for i in range(0,len(myKeys),len(myKeys)//sampleSize):
    sparsity = myKeys[i]
    sparsities.append(sparsity)
    rstructures.append(greater90[sparsity])
    model = RadixNetFuncs.createModel(greater90[sparsity])
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    models.append(model)
# for i in range(len(sparsities)):
#     print(str(sparsities[i])+": "+str(rstructures[i]))

In [9]:
histories = []
strange = []
for i, model in enumerate(models):
    print("model " +str(i+1)+" out of "+str(len(models)))
    epochs = 3
    history = model.fit(X_train,Y_train,epochs = epochs, batch_size = 100,validation_split = 0.1, verbose = 0)
    acc = history.history['accuracy']
    if acc[-1]<=0.75:
        print(str(acc[-1])+" "+str(rstructures[i]))
    histories.append(history)

model 1 out of 21
model 2 out of 21
model 3 out of 21
model 4 out of 21
model 5 out of 21
model 6 out of 21
model 7 out of 21
model 8 out of 21
model 9 out of 21
model 10 out of 21
model 11 out of 21
model 12 out of 21
model 13 out of 21
0.4651851952075958 (((11, 12), (1,)), (6, 3, 1, 1))
model 14 out of 21
model 15 out of 21
0.5723280310630798 (((13, 12), (1,)), (6, 2, 1, 1))
model 16 out of 21
model 17 out of 21
model 18 out of 21
0.5820634961128235 (((8, 15), (2,)), (7, 3, 1, 1))
model 19 out of 21
model 20 out of 21
model 21 out of 21


### Strange models

In [18]:
models = []
sparsities = []
structures = []
file = open("smodels.txt", "r")
lines = file.readlines()
file.close()
desired = [784,300,100,10]
for line in lines:
    line = line.split(" - ")
    sparsities.append(float(line[1]))
    structures.append(literal_eval(line[2]))
    structure = RadixNetFuncs.build_radix_nets([structures[-1][0]], desired)
    #print(structure[0].shape)
    model = RadixNetFuncs.CustomModel(structure)
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    models.append(model)

In [19]:
histories = []
for i, model in enumerate(models):
    print("model " +str(i+1)+" out of "+str(len(models)))
    epochs = 10
    history = model.fit(X_train,Y_train,epochs = epochs, batch_size = 512,validation_split = 0.1, verbose = 0)
    histories.append(history)

model 1 out of 74
model 2 out of 74
model 3 out of 74


KeyboardInterrupt: 

In [ ]:
epochs_range = range(epochs)
plt.figure(figsize=(10, 10))
for i, history in enumerate(histories):
    acc = history.history['accuracy']
    
    plt.plot(epochs_range, acc, label='Sparsity '+str(sparsities[i]))
plt.legend(loc='lower right')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
    
plt.savefig('smodeltraining.png')